In [1]:
%pip install -r "../requirements.txt"

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.21.0%2Bcu126-cp310-cp310-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.6.0%2Bcu126-cp310-cp310-win_amd64.whl.metadata (6.8 kB)
  Using cached https://download.pytorch.org/whl/cu126/torch-2.6.0%2Bcu126-cp310-cp310-win_amd64.whl.metadata (28 kB)
     ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
     ----- ---------------------------------- 0.8/6.2 MB 4.8 MB/s eta 0:00:02
     --------------- ------------------------ 2.4/6.2 MB 5.4 MB/s eta 0:00:01
     ------------------------- -------------- 3.9/6.2 MB 6.0 MB/s eta 0:00:01
     ----------------------------------- ---- 5.5/6.2 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------- 6.2/6.2 MB 6.5 MB/s eta 0:00:00
Using cached https://download.pytorch.org/whl/cu126/torchvision-0.21.0%2Bcu126-cp310-cp310-win_amd64.whl (6.1 MB)
U

  You can safely remove it manually.


In [5]:
!sudo nvidia-smi --gpu-reset -i 0
!nvidia-smi

Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app


Thu Apr 17 20:12:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.02                 Driver Version: 576.02         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   34C    P5             36W /  170W |   11988MiB /  12288MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
import torch

torch.cuda.reset_max_memory_allocated()

In [7]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3060


In [6]:
!pip install transformers==4.40.1 peft==0.4.0
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.



  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
  Using cached peft-0.4.0-py3-none-any.whl.metadata (21 kB)
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.0 MB 7.4 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.0 MB 7.6 MB/s eta 0:00:01
   ------------------- -------------------- 4.5/9.0 MB 7.7 MB/s eta 0:00:01
   ------------------------ --------------- 5.5/9.0 MB 6.7 MB/s eta 0:00:01
   ------------------------------ --------- 6.8/9.0 MB 6.7 MB/s eta 0:00:01
   ---------------------------------- ----- 7.9/9.0 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 9.0/9.0 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------------------------------- ------ 1.8/2.2 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found ex

In [9]:
%pip install dotenv
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# FinGPT Sentiment Analysis

In [3]:
import torch
torch.cuda.empty_cache()


In [9]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [10]:
import torch
import gc

def print_gpu_memory():
    allocated = torch.cuda.memory_allocated() / (1024**2)
    cached = torch.cuda.memory_reserved() / (1024**2)
    print(f"Allocated: {allocated:.2f} MB")
    print(f"Cached: {cached:.2f} MB")

# Before clearing the cache
print("Before clearing cache:")
print_gpu_memory()

# Clearing cache
gc.collect()
torch.cuda.empty_cache()

# After clearing the cache
print("\nAfter clearing cache:")
print_gpu_memory()

Before clearing cache:
Allocated: 18644.61 MB
Cached: 18646.00 MB

After clearing cache:
Allocated: 18644.61 MB
Cached: 18646.00 MB


In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0
import torch
import os
from dotenv import load_dotenv
from transformers import BitsAndBytesConfig

load_dotenv(
    dotenv_path="../.env"
)
huggingface_token = os.getenv("FINGPT_ACCESS_TOKEN")

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, 
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True,
)

# Load Models
base_model = "meta-llama/Meta-Llama-3-8B" 
peft_model = "FinGPT/fingpt-mt_llama3-8b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(
    base_model, 
    trust_remote_code=True, 
    token=huggingface_token,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0")
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()


c:\Users\ARIJIT\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ARIJIT\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\ARIJIT\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\ARIJIT\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_down

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 12.00 GiB of which 0 bytes is free. Of the allocated memory 18.21 GiB is allocated by PyTorch, and 1.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Make prompts
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: '''
]

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512).to(device)
res = model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]


# Show results
for sentiment in out_text:
    print(sentiment)